In [1]:
from pyspark.sql import SparkSession, functions as F
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import col, trim, lower, regexp_replace
import os

In [2]:
path_landing = "../../../delta_lake/csv"
path_creation = "/delta_lake/creation"
path_exploitation = "/delta_lake/exploitation"

In [3]:
mongo_connector_jar = "/home/provira/Documents/TFM/TFM/src/P2/Explotation Zone/jars/mongo-spark-connector_2.12-3.0.1.jar"
mongo_driver_jar = "/home/provira/Documents/TFM/TFM/src/P2/Explotation Zone/jars/mongo-java-driver-3.12.10.jar"

In [4]:
builder = SparkSession.builder \
    .appName("MongoDB-Delta Integration") \
    .config("spark.jars", f"{mongo_connector_jar},{mongo_driver_jar}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

25/06/27 16:12:29 WARN Utils: Your hostname, provira-ERAZER-P6705-MD61203 resolves to a loopback address: 127.0.1.1; using 192.168.1.55 instead (on interface wlo1)
25/06/27 16:12:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/provira/anaconda3/envs/spark_py3.9/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/provira/.ivy2/cache
The jars for the packages stored in: /home/provira/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2ca34fd0-d1c0-4118-9b07-4cb373702694;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 725ms :: artifacts dl 28ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-r

In [5]:
spark.sparkContext._jsc.sc().listJars()


JavaObject id=o53

In [6]:
df_csv = spark.read.format("delta").load(f"{path_landing}")

# Preprocessing

In [7]:
df_csv.columns

['_c0', 'text', 'Emotion']

In [8]:
distinct_emotions = df_csv.select("Emotion").distinct()
distinct_emotions.show()

count_distinct = df_csv.select("Emotion").distinct().count()
total = df_csv.select("Emotion").count()
print(f"Number of distinct Emotion labels: {count_distinct} / {total}")


+--------------------+
|             Emotion|
+--------------------+
|omg!!! loving thi...|
|why the nazis stu...|
| @user as forecas...|
|so simple, but so...|
|   #friday  xoxos...|
|#tgif   #ff to my...|
|@user don't forge...|
|loved that season...|
|@user pay of #ric...|
|#gameshow   bull ...|
|lack of access sp...|
|family 5k @user #...|
|@user fg introduc...|
|@user the library...|
|men's footjoy bla...|
| @user a great #m...|
|happy friday.   #...|
|badminton bareng ...|
|our unknown #futu...|
|my #hea goes out ...|
+--------------------+
only showing top 20 rows



Number of distinct Emotion labels: 78800 / 1100992


In [9]:
# Get count per Emotion
emotion_counts = df_csv.groupBy("Emotion").count()

# Order by count descending and take top 10
top_10_emotions = emotion_counts.orderBy(col("count").desc()).limit(20)

top_10_emotions.show()

+--------------------+------+
|             Emotion| count|
+--------------------+------+
|             neutral|674538|
|                love| 39553|
|           happiness| 27175|
|                null| 18177|
|             sadness| 17491|
|              relief| 16729|
|                hate| 15267|
|               anger| 12356|
|                 fun| 10075|
|          enthusiasm|  9304|
|            surprise|  6954|
|               empty|  5542|
|               worry|  4475|
|           [deleted]|  3861|
|#model   i love u...|   319|
|             boredom|   126|
|              #NAME?|   123|
|        CakeDay--Bot|    96|
|i finally found a...|    82|
|aww yeah it's all...|    75|
+--------------------+------+



In [10]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

#nltk.download('punkt')
#nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Tokenizar
tokenizer = Tokenizer(inputCol="text", outputCol="words")
df_words = tokenizer.transform(df_csv.limit(1000))

# Eliminar stopwords (solo en inglés por defecto, pero puedes pasar las tuyas)
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
remover.setStopWords(list(stop_words))
df_filtered = remover.transform(df_words)

#Stemming
stemmer = SnowballStemmer("english")

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

stem_udf = udf(stem_tokens, ArrayType(StringType()))

df_stemmed = df_filtered.withColumn("stemmed_words", stem_udf("filtered_words"))

df_stemmed.select("text", "filtered_words", "stemmed_words").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                           |filtered_words                                                                                                                                          |stemmed_words                                                                                                                         |
+---

In [11]:
from pyspark.ml.feature import CountVectorizer, IDF

# Paso 1: Crear el CountVectorizer para extraer el vocabulario y conteo de tokens
cv = CountVectorizer(inputCol="stemmed_words", outputCol="raw_features")
cv_model = cv.fit(df_stemmed)             # Entrenas el modelo con el vocabulario
df_featurized = cv_model.transform(df_stemmed)  # Transformas el DataFrame

# Paso 2: Calcular TF-IDF a partir del conteo
idf = IDF(inputCol="raw_features", outputCol="tfidf_features")
idf_model = idf.fit(df_featurized)          # Ajustar IDF sobre los datos
df_tfidf = idf_model.transform(df_featurized) # Transformar con TF-IDF

# Mostrar resultados
df_tfidf.select("stemmed_words", "raw_features", "tfidf_features").show(truncate=False)
df_tfidf.head(1)

+--------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|stemmed_words                                                                                                                         |raw_features                                                                                                                                            

[Row(_c0='0', text='i seriously hate one subject to death but now i feel reluctant to drop it', Emotion='hate', words=['i', 'seriously', 'hate', 'one', 'subject', 'to', 'death', 'but', 'now', 'i', 'feel', 'reluctant', 'to', 'drop', 'it'], filtered_words=['seriously', 'hate', 'one', 'subject', 'death', 'feel', 'reluctant', 'drop'], stemmed_words=['serious', 'hate', 'one', 'subject', 'death', 'feel', 'reluct', 'drop'], raw_features=SparseVector(2383, {0: 1.0, 23: 1.0, 47: 1.0, 172: 1.0, 395: 1.0, 400: 1.0, 538: 1.0, 560: 1.0}), tfidf_features=SparseVector(2383, {0: 0.0243, 23: 3.3824, 47: 3.8642, 172: 4.6062, 395: 5.117, 400: 5.117, 538: 5.5225, 560: 5.5225}))]

## Eliminación de puntuación


In [12]:
df_csv_clean = df_csv.dropna() \
    .withColumnRenamed("_c0", "id") \
    .withColumnRenamed("Emotion", "emotion") \
    .withColumn("text", trim(col("text"))) \
    .withColumn("text", lower(col("text"))) \
    .withColumn("text", regexp_replace(col("text"), r"\bi m\b", "i'm")) \
    .withColumn("text", regexp_replace(col("text"), r"[^a-zA-Z0-9\s']", ""))  # keep letters, digits, spaces, apostrophes

df_csv_clean.head(5)

[Row(id='0', text='i seriously hate one subject to death but now i feel reluctant to drop it', emotion='hate'),
 Row(id='1', text='im so full of life i feel appalled', emotion='neutral'),
 Row(id='2', text='i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it', emotion='neutral'),
 Row(id='3', text='ive been really angry with r and i feel like an idiot for trusting him in the first place', emotion='anger'),
 Row(id='4', text='i feel suspicious if there is no one outside like the rapture has happened or something', emotion='neutral')]

In [13]:
# Get count per Emotion
emotion_counts = df_csv_clean.groupBy("emotion").count()

# Order by count descending and take top 10
top_10_emotions = emotion_counts.orderBy(col("count").desc()).limit(20)

top_10_emotions.show()

emotion_names = [row['emotion'] for row in top_10_emotions.collect()]


+--------------------+------+
|             emotion| count|
+--------------------+------+
|             neutral|674538|
|                love| 39553|
|           happiness| 27175|
|             sadness| 17491|
|              relief| 16729|
|                hate| 15267|
|               anger| 12356|
|                 fun| 10075|
|          enthusiasm|  9304|
|            surprise|  6954|
|               empty|  5542|
|               worry|  4475|
|           [deleted]|  3861|
|#model   i love u...|   319|
|             boredom|   126|
|              #NAME?|   123|
|        CakeDay--Bot|    96|
|i finally found a...|    82|
|aww yeah it's all...|    75|
|       devildriver77|    74|
+--------------------+------+



# Store in MongoDB

In [ ]:
df_tfidf.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- text: string (nullable = true)
 |-- Emotion: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stemmed_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- raw_features: vector (nullable = true)
 |-- tfidf_features: vector (nullable = true)



In [20]:

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import VectorUDT

def vector_to_array(v):
    return v.toArray().tolist() if v else None

vector_to_array_udf = udf(vector_to_array, ArrayType(FloatType()))

df_tfidf_safe = df_tfidf \
    .withColumn("raw_features_array", vector_to_array_udf("raw_features")) \
    .withColumn("tfidf_features_array", vector_to_array_udf("tfidf_features"))

df_tfidf_safe.select(
    "text", "Emotion", "words", "filtered_words", "stemmed_words",
    "raw_features_array", "tfidf_features_array"
).write \
    .format("mongo") \
    .option("uri", "mongodb://localhost:27017") \
    .option("database", "tfm") \
    .option("collection", "tf-idf") \
    .mode("append") \
    .save()